In [1]:
sc

In [2]:
import operator
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#FINDING SPECIAL DATES AND YELP REVIEWS

In [4]:
def extractRow(index, lines):
    import csv
    if index == 0:
        lines.next()
        
    reader = csv.reader(lines)
    for row in reader:
        yield row

In [5]:
reviews_rdd = sc.textFile("yelp_review.csv", use_unicode=False)
reviews_rdd.getNumPartitions()
reviews_rdd_new = reviews_rdd.mapPartitionsWithIndex(extractRow).cache()


In [6]:
reviews_rdd_new.take(5)[4][0]

'L_9BTb55X0GDtThi6GlZ6w'

In [7]:
# def countRow(filename):
#     import csv
#     count = 0
#     with open(filename, 'rb') as fi:
#         reader = csv.reader(fi)
        
#         for row in reader:
#             count += 1
        
#     print count

# countRow('yelp_review.csv')

In [8]:
#reviews_rdd_new = reviews_rdd_new.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7]))
reviews_rdd_newClear = reviews_rdd_new.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]))

In [9]:
from pyspark.sql.types import *
#Convert to dataframe
reviews_df = reviews_rdd_newClear.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8])).toDF()
reviews_df = reviews_df.select(reviews_df['_1'].alias('ReviewID'), 
                               reviews_df['_2'].alias('UserID'),
                               reviews_df['_3'].alias('BusinessID'), 
                               reviews_df['_4'].alias('Stars').cast(IntegerType()),
                               reviews_df['_5'].alias('Date').cast(DateType()), 
                               reviews_df['_6'].alias('Text'),
                               reviews_df['_7'].alias('Useful').cast(IntegerType()), 
                               reviews_df['_8'].alias('Funny').cast(IntegerType()),
                               reviews_df['_9'].alias('Cool').cast(IntegerType()))

In [10]:
reviews_df.show(4)

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|            ReviewID|              UserID|          BusinessID|Stars|      Date|                Text|Useful|Funny|Cool|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|     0|    0|   0|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|    5|2016-05-28|Small unassuming ...|     0|    0|   0|
|MV3CcKScW05u5LVfF...|bv2nCi5Qv5vroFiqK...|CKC0-MOWMqoeWf6s-...|    5|2016-05-28|Lester's is locat...|     0|    0|   0|
|IXvOzsEMYtiJI0CAR...|bv2nCi5Qv5vroFiqK...|ACFtxLv8pGrrxMm6E...|    4|2016-05-28|Love coming here....|     0|    0|   0|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+
only showing top 4 rows



In [11]:
#START QUERIES

In [12]:
#Get Queries for Valentines Day and What Business
valentines_df = reviews_df.select(reviews_df["BusinessID"],reviews_df["Date"],reviews_df["Text"],reviews_df["Stars"])

#valentines_df = valentines_df.filter(valentines_df.Date == '2016-02-14')
valentines_dates = []
for year in range(4,18):
    valentines_dates.append('20' + ("0"+str(year) if year < 10 else str(year)) + '-02-14')
valentines_df = valentines_df.where(valentines_df.Date.isin(valentines_dates))

valentines_df.show(3)

+--------------------+----------+--------------------+-----+
|          BusinessID|      Date|                Text|Stars|
+--------------------+----------+--------------------+-----+
|FK6XNUqPCdfgU0-wu...|2015-02-14|Wow!!!!!!! Mon rÃ...|    5|
|fYaSaFxc79Db1EBhf...|2011-02-14|This hotel has a ...|    4|
|lV840VDGdQUS78LPV...|2016-02-14|I found ABC Auto ...|    5|
+--------------------+----------+--------------------+-----+
only showing top 3 rows



In [13]:
#valentines_business_count = valentines_df.select(valentines_df['BusinessID'])
val_one = valentines_df.select(valentines_df['BusinessID'])
#valentines_business_count = valentines_df.select(valentines_df['BusinessID'])
val = val_one.na.drop(subset=['`BusinessID`'])
val.show()
val = val.groupBy('BusinessID').count()
val.show()
#valentines_business_count = valentines_business_count.sort('Count', ascending=False)

+--------------------+
|          BusinessID|
+--------------------+
|FK6XNUqPCdfgU0-wu...|
|fYaSaFxc79Db1EBhf...|
|lV840VDGdQUS78LPV...|
|1a9YViZDThy3VqBnG...|
|5AjPvB-PKBv5YP3JN...|
|LelAlfuj5oVRF9CQd...|
|se9MhDAQjACYMO3LC...|
|qmIHO-6T_KEfPC9jy...|
|wUrnir4yQj4Lnta-j...|
|KjicU7uxRt2KDEnO5...|
|euksvHm653hJMJBqz...|
|VicoxAai5JINrWwVw...|
|1Vvxx-9AoINg8TJXO...|
|ZosrzTfiiPhbURA04...|
|Cni2l-VKG_pdospJ6...|
|7Xw_vv-ekUL2sTqip...|
|br2M2b-CsXI0HY0RX...|
|N93EYZy9R0sdlEvub...|
|qkDyVDlbTbANNK307...|
|IKqVo1skSa2aubLfc...|
+--------------------+
only showing top 20 rows



Py4JJavaError: An error occurred while calling o156.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 5.0 failed 1 times, most recent failure: Lost task 3.0 in stage 5.0 (TID 8, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 174, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 169, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-8-19b13de378a5>", line 2, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:156)
	at org.apache.spark.api.python.PythonRunner$$anon$1.next(PythonRDD.scala:152)
	at org.apache.spark.Interrup

In [ ]:
#valentines_bussines_count.count()
#valentines_business_count.show()
#valentines_business_count.take(2)
print(type(val))
val.show()